In [10]:
import pandas as pd

def read_json(path = 'data/obfuscated_data_06.json'):
    df = pd.read_json(path, orient="records",encoding='utf-8')
    return df

df = read_json()

In [5]:
# !pip install azure-ai-textanalytics

  Using cached azure_ai_textanalytics-5.3.0-py3-none-any.whl.metadata (82 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl.metadata (5.0 kB)
Using cached azure_ai_textanalytics-5.3.0-py3-none-any.whl (298 kB)
Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [1]:
import json

with open('input/secret.json', 'r', encoding='UTF-8') as file:
    secret_json = json.load(file)
language_key = secret_json['LANGUAGE_KEY']
language_endpoint = secret_json['LANGUAGE_ENDPOINT']

### Tried Python first (Synchronous requests, doesn't work)
#### Please jump to Section "Switched to using REST API (Asynchronous requests)"

In [3]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(endpoint=language_endpoint, credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [48]:
import ast
with open('data/test_indices_2.txt', 'r') as file:
    test_indices_2 = text_indices_2 = ast.literal_eval(file.read().strip())

exceed_5120 = []
max_len = 0
max_idx = 0
for i in range (len(df)):
    curr_document = df.iloc[i].document
    if curr_document in test_indices_2:
        curr_len = len(df.iloc[i].full_text)
        if curr_len > 5120:
            if curr_len > max_len:
                max_len = curr_len
                max_idx = curr_document
            exceed_5120.append(curr_document)

print(exceed_5120)
print(f"{len(exceed_5120)} out of {len(test_indices_2)} test files have char length larger than 5120 characters")
print(f"Maximum char length: {max_len} of document #{max_idx}")

[18, 56, 67, 89, 91, 94, 113, 156, 176, 185, 186, 192, 194, 199, 213, 241, 250, 264, 270, 278, 289, 295, 320, 330, 331, 334, 342, 360, 362, 363, 364, 394, 411, 422, 433, 454, 464, 472, 481, 505, 542, 547, 572, 576, 586, 589, 616, 642, 650, 653, 682, 716, 720, 726, 729, 751, 756, 760, 769, 776, 778, 783, 801, 823, 838, 844, 850, 858, 859, 894, 911, 921, 930, 931, 944, 955, 965, 971, 976, 980, 993, 994, 1010, 1014, 1030, 1042, 1043, 1050, 1053, 1082, 1084, 1121, 1125, 1136, 1151, 1154, 1161, 1168, 1170, 1175, 1181, 1185, 1200, 1213, 1214, 1240, 1242, 1262, 1281, 1284, 1285, 1324, 1358, 1359, 1363, 1364, 1366, 1385, 1393, 1394, 1422, 1425, 1438, 1456, 1518, 1519, 1521, 1527, 1532, 1541, 1546, 1582, 1606, 1649, 1663, 1701, 1707, 1751, 1761, 1765, 1766, 1782, 1795, 1799, 1802, 1816, 1832, 1846, 1863, 1930, 1960, 1961, 1962, 1983, 1992, 1999, 2005, 2006, 2018, 2028, 2031, 2035, 2040, 2050, 2052, 2058, 2089, 2113, 2142, 2143, 2145, 2150, 2162, 2180, 2186, 2199, 2203, 2220, 2222, 2226, 2227, 2

In [33]:
# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client):
    # documents = [
    #     "The employee's SSN is 859-98-0987.",
    #     "The employee's phone number is 555-555-5555."
    # ]
    # documents = [df.iloc[2672].full_text, df.iloc[5257].full_text]
    documents = [df.iloc[5257].full_text]
    response = client.recognize_pii_entities(documents, language="en")
    for doc in response:
        print(doc)
    # result = [doc for doc in response if not doc.is_error]
    
    # i = 0
    # for doc in result:
    #     # print("Redacted Text: {}".format(doc.redacted_text))
    #     print(f"Iteration {i} ----------------")
    #     i += 1
    #     for entity in doc.entities:
    #         print("Entity: {}".format(entity.text))
    #         print("\tCategory: {}".format(entity.category))
    #         print("\tConfidence Score: {}".format(entity.confidence_score))
    #         print("\tOffset: {}".format(entity.offset))
    #         print("\tLength: {}".format(entity.length))
pii_recognition_example(client)

{'id': '0', 'error': TextAnalyticsError(code=InvalidDocument, message=A document within the request was too large to be processed. Limit document size to: 5120 text elements. For additional details on the data limitations see https://aka.ms/text-analytics-data-limits, target=None), 'is_error': True, 'kind': 'DocumentError'}


In [16]:
def detect_pii(texts):
    response = client.recognize_pii_entities(texts)
    results = [result for result in response if not result.is_error]
    return results

input_texts = [df.iloc[2672].full_text, df.iloc[5257].full_text]
# input_texts = ["Hi, my name is Joe Doe. My phone number is 555-124-5263. Please email me at joedoe@hotmail.com."]  # List of all 22,688 text inputs
pii_results = detect_pii(input_texts)


In [17]:
print(len(pii_results))

1


In [21]:
for doc in pii_results:
    for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))
    # print(doc.entities)

Entity: Asif Khan
	Category: Person
	Confidence Score: 1.0
	Offset: 50
	Length: 9
Entity: January 16, 2021
	Category: DateTime
	Confidence Score: 1.0
	Offset: 93
	Length: 16
Entity: users
	Category: PersonType
	Confidence Score: 0.97
	Offset: 625
	Length: 5
Entity: people
	Category: PersonType
	Confidence Score: 0.69
	Offset: 917
	Length: 6
Entity: Food
	Category: PersonType
	Confidence Score: 0.71
	Offset: 925
	Length: 4
Entity: makers
	Category: PersonType
	Confidence Score: 0.42
	Offset: 930
	Length: 6
Entity: delivery persons
	Category: PersonType
	Confidence Score: 0.85
	Offset: 938
	Length: 16
Entity: users
	Category: PersonType
	Confidence Score: 0.98
	Offset: 960
	Length: 5
Entity: http://jacobs-fisher.com/listpost.html)
	Category: URL
	Confidence Score: 0.8
	Offset: 1312
	Length: 39
Entity: users
	Category: PersonType
	Confidence Score: 0.95
	Offset: 1474
	Length: 5
Entity: stakeholders
	Category: PersonType
	Confidence Score: 0.98
	Offset: 1488
	Length: 12
Entity: now
	Catego

### Switched to using REST API (Asynchronous requests)

In [111]:
import json
import requests

with open('input/secret.json', 'r', encoding='UTF-8') as file:
    secret_json = json.load(file)
language_key = secret_json['LANGUAGE_KEY']
language_endpoint = secret_json['LANGUAGE_ENDPOINT']
# endpoint = language_endpoint + 'language/:analyze-text?api-version=2023-04-01'
endpoint = language_endpoint + 'language/:analyze-text?api-version=2023-04-15-preview'

In [107]:
headers = {
    'Ocp-Apim-Subscription-Key': language_key,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

In [115]:
# Create the Asynchronous Request
def create_pii_detection_job(text):
    data = {
                "kind": "PiiEntityRecognition",
                "parameters": {
                    "modelVersion": "latest",
                    "piiCategories": ["Person", "URL", "Email", "PhoneNumber"]
                },
                "analysisInput":{
                    "documents":[
                        {
                            "id":"1",
                            "language": "en",
                            "text": text
                        }
                    ]
                }
            }

    response = requests.post(endpoint, headers=headers, json=data)
    return response.json()
    # if response.status_code == 202:
    #     print("Job created successfully. Job ID:", response.headers["operation-location"])
    #     return response.headers["operation-location"]
    # else:
    #     print("Error:", response.json())
    #     return None


In [63]:
# # Poll for the Job’s Status
# import time

# def get_job_status(operation_location):
#     while True:
#         response = requests.get(operation_location, headers=headers)
#         result = response.json()

#         if response.status_code == 200:
#             if result['status'] == 'succeeded':
#                 print("Job completed successfully!")
#                 return result['tasks']['items']
#             elif result['status'] in ['failed', 'cancelled']:
#                 print("Job failed or was cancelled.")
#                 return None
#             else:
#                 print("Job in progress... checking again in 10 seconds.")
#         else:
#             print("Error:", result)
#             return None
        
#         time.sleep(10)


In [116]:
# Process the Results
def process_results(items):
    for item in items:
        pii_entities = item.get("results", {}).get("documents", [])[0].get("entities", [])
        for entity in pii_entities:
            print(f"Detected PII entity: {entity['text']}, Category: {entity['category']}, Confidence: {entity['confidenceScore']}")

def process_results(response):
    # Extracting detected entities from the response
    for document in response['results']['documents']:
        redacted_text = document['redactedText']
        print(f"Redacted Text: {redacted_text}")
        
        for entity in document['entities']:
            print(f"Detected PII Entity: {entity['text']}")
            print(f"Category: {entity['category']}, Confidence: {entity['confidenceScore']}\n")

# Example usage:
input_file = df.iloc[2672].full_text
# input_file = df.iloc[5257].full_text
json_response = create_pii_detection_job(input_file)
process_results(json_response)
# if job_id:
#     job_results = get_job_status(job_id)
#     if job_results:
#         process_results(job_results)


Redacted Text: Visualization

Peer-graded Assignment: Refection

*********

Design Thinking for Innovation

January 16, 2021

Challenge

We all love to eat good food. Cooking good food at home is not always feasible.  Sometimes we want to taste diferent good food from diferent sources. We visit our  nearest restaurants, food shops, travel to diferent regions, and taste diferent cuisine.  We also love to have good food delivered to their homes.

In the COVID-19 pandemic, the situation is changed. The demand for food delivery  services is geting higher. we use the tool of Visualization, or Visual Thinking, to  understand and know the users' pain points, come up with user-friendly and  functional solutions.

Selection

Visualization is one of the very important tools. It is a core component of the way to  communicate. It allows us to think in a nonverbal way. Designing a food delivery service  involves diferent groups of people. Food makers, delivery persons, and users of the  service are

In [117]:
json_response

{'kind': 'PiiEntityRecognitionResults',
 'results': {'documents': [{'redactedText': 'Visualization\n\nPeer-graded Assignment: Refection\n\n*********\n\nDesign Thinking for Innovation\n\nJanuary 16, 2021\n\nChallenge\n\nWe all love to eat good food. Cooking good food at home is not always feasible.  Sometimes we want to taste diferent good food from diferent sources. We visit our  nearest restaurants, food shops, travel to diferent regions, and taste diferent cuisine.  We also love to have good food delivered to their homes.\n\nIn the COVID-19 pandemic, the situation is changed. The demand for food delivery  services is geting higher. we use the tool of Visualization, or Visual Thinking, to  understand and know the users\' pain points, come up with user-friendly and  functional solutions.\n\nSelection\n\nVisualization is one of the very important tools. It is a core component of the way to  communicate. It allows us to think in a nonverbal way. Designing a food delivery service  involve

In [122]:
df.iloc[5257].full_text

'Design Thinking for Innovation - Assignment\n\nSherif Mackie (India)           Mobile Number:    764.356.3313x9314\n\nLinkedIn:       Email:  http://www.linkedin.com/in/catherinemoore    peter93@hotmail.com\n\nChallenge – Develop An Organizational Health Card\n\nDr. Angie’s Foundation (angiesfoundation.org/) is a 20 year old organization in the Social\n\nDevelopment space that operates various programmes in around 20 states across India. Over the past  couple of years, the organization has undertaken a transformational journey and is currently looking to set  itself up as an impactful force driving innovative solutions to adaptive social problems.\n\nThe challenge was to create a score card which could be used to measure the entire organization\n\nnot only by the activities and deliverables but also in terms of its inherent robustness and preparedness to  move to the next level. This is the first time the organization is attempting a monitoring and evaluation  mechanism at this scale.